# Detect Anomalies Using Density Based Clustering


## Clustering-Based Anomaly Detection

- Assumption: Data points that are similar tend to belong to similar groups or clusters, as determined by their distance from local centroids. Normal data points occur around a dense neighborhood and abnormalities are far away.

- Using density based clustering, like DBSCAN, we can design the model such that the data points that do not fall into a cluster are the anomalies.

https://docs.google.com/presentation/d/1frjnm1ii63gcFSyhQxxzvekrMAAO0xnw-aitdTPIbc0/edit?usp=sharing

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

import env

In [ ]:
def get_curriculum_logs():
    filename = "curriculum-access.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=False)
    else:
        # read the SQL query into a dataframe
        url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
        query = '''
        SELECT date,
               path as endpoint,
               user_id,
               cohort_id,
               ip as source_ip
        FROM logs;
        '''
        df = pd.read_sql(query, url)

        # Write that dataframe to disk for later.
        df.to_csv(filename, index = False)

        return df  

In [ ]:
# acquire data using the above function
df = get_curriculum_logs()
df.head()

In [ ]:
# convert date to a pandas datetime format and set as index
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

Who is accessing the curriculum a lot (total views) and possibiliy looking at lot of unique pages (scraping?) 

Aggregate and compute 2 features...number of unique pages and total page views. 

In [ ]:
df.head()

In [ ]:
page_views = df.groupby(['user_id'])['endpoint'].agg(['count', 'nunique'])
page_views

Scale each attribute linearly. 

In [ ]:
# create the scaler
scaler = MinMaxScaler().fit(page_views)
# use the scaler
page_views_scaled_array = scaler.transform(page_views)
page_views_scaled_array[0:10]

Construct a DBSCAN object that requires a minimum of 4 data points in a neighborhood of radius 0.1 to be considered a core point.

In [ ]:
page_views.shape

In [ ]:
dbsc = DBSCAN(eps = 0.1, min_samples=4).fit(page_views_scaled_array)
print(dbsc)

In [ ]:
# Now, let's add the scaled value columns back onto the dataframe

columns = list(page_views.columns)
scaled_columns = ["scaled_" + column for column in columns]
scaled_columns

In [ ]:
# Create a dataframe containing the scaled values
scaled_df = pd.DataFrame(page_views_scaled_array, columns=scaled_columns, index=page_views.index)
scaled_df.head()

In [ ]:
page_views.head()

In [ ]:
# Merge the scaled and non-scaled values into one dataframe
page_views = page_views.merge(scaled_df, left_index=True, right_index=True)
page_views

In [ ]:
labels = dbsc.labels_

In [ ]:
labels[0:15]

#### label '-1' represents observations which are not part of any cluster and are potential anomalies warranting further investigation

In [ ]:
#add labels back to the dataframe
page_views['labels'] = labels

# how many unique labels (clusters) are created by DBSCAN?
page_views.labels.value_counts()

In [ ]:
page_views[page_views.labels==-1]

In [ ]:
plt.scatter(page_views['scaled_count'], page_views['scaled_nunique'], c=page_views.labels)
plt.show()

## Experiment with the DBSCAN properties
- Read up on the epsilon and min_samples arguments into DBSCAN at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
- Experiment with altering the epsilon values (the `eps` argument holding the threshhold parameter). Run the models and visualize the results. What has changed? Why do you think that is?
- Double the `min_samples` parameter. Run your model and visualize the results. Consider what changed and why.

# Exercise

**file name:** clustering_anomaly_detection.py or clustering_anomaly_detection.ipynb


### Clustering - DBSCAN

Ideas: 

Use DBSCAN to detect anomalies in curriculumn access. 

Use DBSCAN to detect anomalies in other products from the customers dataset. 

Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).
